# Maxwell-Stefan diffusion 

In [ ]:
!pip install fipy

In [ ]:
from fipy import CellVariable, Grid1D, Grid2D, PeriodicGrid1D
from fipy import DiffusionTerm, TransientTerm
from fipy import Viewer
from numpy import log, exp


In [ ]:
tend = 1.0
dt = 1.0e-2
D01 = 1.0
D02 = 1.0
D12 = 1.0

# thermodynamic parameters
chi01 = 0.0
chi02 = 0.0
chi12 = 0.0

vv = 1.0  # reference volume: all volumes will be in units of vv
v0 = 1.0
v1 = 1.0
v2 = 1.0


phi_solute = 0.1


In [ ]:
nx = 128
Lx = 1.0
mesh = Grid1D(nx=nx, Lx=Lx)
#mesh = PeriodicGrid1D(nx=nx, dx=dx)

#nx = 20
#ny = 20
#Lx = 1.0
#Ly = 1.0
#mesh = Grid2D(Lx=Lx, Ly=Ly, nx=nx, ny=ny)


x_mesh = mesh.cellCenters


################################################################################
phi1 = CellVariable(mesh=mesh, hasOld=True, value=0.0, name='solvent')
phi2 = CellVariable(mesh=mesh, hasOld=True, value=0.0, name='nonsolvent')
phi0 = CellVariable(mesh=mesh, hasOld=True, value=0.0, name='solute')
c = CellVariable(mesh=mesh, hasOld=True, value=0.0)
mu0 = CellVariable(mesh=mesh, hasOld=True, value=0.0)
mu1 = CellVariable(mesh=mesh, hasOld=True, value=0.0)
mu2 = CellVariable(mesh=mesh, hasOld=True, value=0.0)


In [ ]:
SMALL = 1.0e-6
phi1.setValue(1.0-phi_solute-SMALL, where=(x_mesh[0] <= 0.5))
phi1.setValue(SMALL, where=(x_mesh[0] > 0.5))
phi2.setValue(1.0-SMALL, where=(x_mesh[0] > 0.5))
phi2.setValue(SMALL, where=(x_mesh[0] <= 0.5))
phi0.value = 1.0 - phi1.value - phi2.value


viewer = Viewer(vars=(phi1, phi2, phi0),  datamin=-0.1, datamax=1.1)


In [ ]:
x1 = phi1/v1 / (phi0/v0 + phi1/v1 + phi2/v2)
x2 = phi2/v2 / (phi0/v0 + phi1/v1 + phi2/v2)
x0 = 1.0 - x1 - x2

Delta = x1/(D12*D01) + x2/(D12*D02) + x0/(D01*D02)
dd11 = ((1.0-phi1)*v1/D02 + phi1*v0/D12) / Delta 
dd12 = - phi1*(v2/D01 - v0/D12) / Delta 
dd21 = - phi2*(v1/D02 - v0/D12) / Delta 
dd22 = ((1.0-phi2)*v2/D01 + phi2*v0/D12) / Delta 

# x_\alpha\frac{\partial\mu_\alpha}{\partial x_{\alpha'}}
# c_\alpha\frac{\partial\mu_\alpha}{\partial \phi_{\alpha'}}
dmu11 = 1.0/v1 + phi1*(-(1.0/v1-1.0/v0) + (chi01+chi02-chi12)*phi2 - 2.0*chi01*(1.0-phi1)) 
dmu12 = 0.0    + phi1*(-(1.0/v2-1.0/v0) + (chi12-chi01-chi02)*(1.0-phi1) + 2.0*chi02*phi2) 
dmu21 = 0.0    + phi2*(-(1.0/v1-1.0/v0) + (chi12-chi01-chi02)*(1.0-phi2) + 2.0*chi01*phi1) 
dmu22 = 1.0/v2 + phi2*(-(1.0/v2-1.0/v0) + (chi01+chi02-chi12)*phi1 - 2.0*chi02*(1.0-phi2)) 

DD11 = dd11*dmu11+dd12*dmu21
DD12 = dd11*dmu12+dd12*dmu22
DD21 = dd21*dmu11+dd22*dmu21
DD22 = dd22*dmu22+dd21*dmu12


#eq1 = TransientTerm(var=x1) == DiffusionTerm(coeff=1.0, var=x1)
#eq2 = TransientTerm(var=x2) == DiffusionTerm(coeff=1.0, var=x2)
eq1 = TransientTerm(var=phi1) == DiffusionTerm(coeff=DD11, var=phi1) + DiffusionTerm(coeff=DD12, var=phi2)
eq2 = TransientTerm(var=phi2) == DiffusionTerm(coeff=DD21, var=phi1) + DiffusionTerm(coeff=DD22, var=phi2)

eq = eq1 & eq2 


In [ ]:
def set_variables(phi1, phi2):
    
    phi0.value = 1.0 - phi1 - phi2
    mu0.value = phi0*exp((1.0-v0/v1)*phi1 + (1.0-v0/v2)*phi2 + v0*((chi01*phi1+chi02*phi2)*(phi1+phi2)-chi12*phi1*phi2))
    mu1.value = phi1*exp((1.0-v1/v0)*(1.0-phi1) - (v1/v2-v1/v0)*phi2 + v1*((chi12-chi01-chi02)*(1.0-phi1)*phi2+chi01*(1.0-phi1)**2+chi02*phi2**2))
    mu2.value = phi2*exp((1.0-v2/v0)*(1.0-phi2) - (v2/v1-v2/v0)*phi1 + v2*((chi12-chi01-chi02)*(1.0-phi2)*phi1+chi02*(1.0-phi2)**2+chi01*phi1**2))


In [ ]:
count = 0
t = 0.0

while (t < tend):

    phi1.updateOld()
    phi2.updateOld()
    eq.solve(dt=dt)

    set_variables(phi1, phi2)

    count += 1
    t += dt
    viewer.plot()
